# SQLite3 naudojimas Python aplinkoje


# SQLite privalumai

SQLite yra lengva, failu paremta duomenų bazė, kuri nereikalauja atskiro serverio. Ji puikiai tinka mažiems projektams, prototipavimui, mokymuisi ir analizės demonstracijoms.

**Pagrindiniai privalumai:**
- Nereikia instaliuoti ar valdyti serverio – visa duomenų bazė yra viename `.db` faile.
- Paprasta naudoti – veikia iš karto su `sqlite3` moduliu Python'e.
- Puikiai tinka darbui su `pandas` ir interaktyviems `Jupyter` projektams.
- Naudojama daugelyje mobiliųjų, naršyklinių ir įtaisytų sistemų kaip vidinė saugykla.

SQLite yra puikus pasirinkimas, kai reikia greito ir paprasto sprendimo be infrastruktūros sudėtingumo.

Pagrindiniai žingsniai, kaip naudoti SQLite duomenų bazę su Python kalba naudojant `sqlite3` modulį.

**1. Prisijungimas prie SQLite duomenų bazės**

Naudojama funkcija `sqlite3.connect('pavyzdys.db')`, kuri sukuria ar atidaro failą pavadinimu `pavyzdys.db`. Sukuriamas ir `cursor`, kuris vykdo SQL komandas.

**2. Lentelės kūrimas**

Norėdami sukurti lentelę, naudojame `cursor.execute()` funkciją su SQL užklausa. Pavyzdžiui, sukurti lentelę `darbuotojai` su stulpeliais `ID`, `Vardas`, `Amzius`, `Departamentas`. Naudojamas `IF NOT EXISTS`, kad nekiltų klaida, jei lentelė jau egzistuoja.

**3. Duomenų įterpimas**

Funkcija `cursor.executemany()` leidžia įterpti kelias eilutes vienu metu. Reikšmės perduodamos kaip sąrašas `tuples`. `?` žymekliai naudojami vietoje tiesioginių reikšmių – tai padeda išvengti SQL injekcijų.

**4. Duomenų nuskaitymas**

Norint nuskaityti duomenis, naudojame `cursor.execute("SELECT * FROM darbuotojai")`, o po to `cursor.fetchall()`, kuris grąžina visus rezultatus kaip sąrašą.

**5. Naudojimas su pandas**

Galime naudoti `pd.read_sql("SELECT * FROM darbuotojai", conn)` – tai leidžia SQL užklausos rezultatą nuskaityti tiesiai į pandas `DataFrame`.

**6. Pakeitimų išsaugojimas ir uždarymas**

Baigus darbą būtina išsaugoti pakeitimus naudojant `conn.commit()`, o po to uždaryti ryšį su `conn.close()`.

**Santrauka:** SQLite yra lengvai naudojama failinė duomenų bazė, kuri nereikalauja serverio. Ji tinka testavimui, praktinėms užduotims ir nedideliems projektams.


In [1]:
import sqlite3
import pandas as pd

# Prisijungimas prie SQLite duomenų bazės (jei tokios nėra, ji bus sukurta)
conn = sqlite3.connect('pavyzdys.db')
cursor = conn.cursor()

cursor.execute('''
DROP TABLE IF EXISTS darbuotojai
''')

# Sukuriame lentelę (jei jos nėra)
cursor.execute('''
CREATE TABLE IF NOT EXISTS darbuotojai (
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    Vardas TEXT NOT NULL,
    Amzius INTEGER NOT NULL,
    Departamentas TEXT NOT NULL
)
''')

# Įdedame kai kuriuos duomenis
cursor.executemany('''
INSERT INTO darbuotojai (Vardas, Amzius, Departamentas)
VALUES (?, ?, ?)
''', [
    ('Jonas', 30, 'Administracija'),
    ('Toma', 35, 'Administracija'),
    ('Marius', 25, 'IT'),
    ('Simas', 21, 'IT'),
    ('Marija', 40, 'IT'),
    ('Gabija', 27, 'IT'),
    ('Simona', 28, 'Finansai'),
    ('Gintarė', 45, 'Finansai'),
    ('Paulius', 38, 'Finansai'),
    ('Saulius', 35, 'Gamyba'),
    ('Gintaras', 29, 'Gamyba'),
    ('Darius1', 31, 'Gamyba'),
    ('Darius2', 31, 'Gamyba'),
    ('Darius3', 31, 'Gamyba'),
    ('Darius4', 31, 'Gamyba')
])

# Įvykdome pakeitimus ir uždarome prisijungimą
conn.commit()

# Kas yra cursor Python'e dirbant su SQL?

`cursor` – tai objektas, kuris naudojamas vykdyti SQL komandas prisijungus prie duomenų bazės. Jis sukuriamas iš duomenų bazės prisijungimo objekto naudojant `conn.cursor()`. Dirbant su `sqlite3`, `pymysql` ar `mysql.connector`, cursor veikia kaip tarpininkas tarp Python kodo ir SQL serverio.

Per `cursor` galima vykdyti įvairias SQL užklausas: `SELECT`, `INSERT`, `UPDATE`, `DELETE`, `CREATE TABLE` ir kt. Pavyzdžiui, norint nuskaityti duomenis iš lentelės, rašoma: `cursor.execute("SELECT * FROM darbuotojai")`, o norint gauti rezultatus – naudojama `cursor.fetchall()`.

Vienu metu galima turėti vieną aktyvią užklausą su tuo pačiu cursor. Baigus darbą, galima (nors ne visada būtina) uždaryti cursor naudojant `cursor.close()`.

Apibendrinant, `cursor` yra būtinas objektas, kai dirbame su SQL iš Python – jis leidžia vykdyti komandas, gauti rezultatus ir palaiko ryšį tarp Python ir duomenų bazės.


In [2]:
query = '''
SELECT * FROM darbuotojai
'''

# Vykdome užklausą ir užpildome parametrus su kintamaisiais
cursor.execute(query)

# Gauname rezultatus
results = cursor.fetchall()

# Paverčiame rezultatus į pandas DataFrame
df = pd.DataFrame(results, columns=['ID', 'Vardas', 'Amzius', 'Departamentas'])

print(df)

    ID    Vardas  Amzius   Departamentas
0    1     Jonas      30  Administracija
1    2      Toma      35  Administracija
2    3    Marius      25              IT
3    4     Simas      21              IT
4    5    Marija      40              IT
5    6    Gabija      27              IT
6    7    Simona      28        Finansai
7    8   Gintarė      45        Finansai
8    9   Paulius      38        Finansai
9   10   Saulius      35          Gamyba
10  11  Gintaras      29          Gamyba
11  12   Darius1      31          Gamyba
12  13   Darius2      31          Gamyba
13  14   Darius3      31          Gamyba
14  15   Darius4      31          Gamyba


# SQL užklausos vykdymas ir rezultatų konvertavimas į DataFrame

Kintamajame `query` aprašoma SQL užklausa kaip tekstas: `SELECT * FROM darbuotojai`. Ji perduodama vykdymui per `cursor.execute(query)`. Tai įvykdo komandą duomenų bazėje ir leidžia gauti rezultatus.

Funkcija `cursor.fetchall()` surenka visus rezultatus į sąrašą. Gautas sąrašas paverčiamas į `pandas` DataFrame naudojant `pd.DataFrame(results, columns=...)`, kur stulpelių pavadinimai turi atitikti SQL lentelės struktūrą.

Galutinis rezultatas – tvarkingas duomenų rinkinys Python'e, kurį galima analizuoti ar vizualizuoti su `pandas`.


In [3]:
#Eksportuoti SQLite duomenis į CSV
df.to_csv("darbuotojai.csv", index=False)

# Ar galima SQLite `.db` failą naudoti MySQL Workbench?

Tiesiogiai – ne. SQLite ir MySQL yra skirtingos duomenų bazių valdymo sistemos. SQLite `.db` failas yra vienas binarinis failas, skirtas SQLite varikliui. Tuo tarpu MySQL Workbench veikia tik su MySQL serveriu, kuris turi visiškai kitokią struktūrą ir formatą.

## Ką galima daryti vietoj to?

**1. Eksportuoti SQLite lenteles į CSV failus:**  
Naudojant `pandas` galima eksportuoti duomenis su `df.to_csv("failas.csv", index=False)`. Tada CSV galima importuoti į MySQL per Workbench naudodami "Table Data Import Wizard".

**2. Eksportuoti SQLite duomenis į SQL dump:**  
Komanda `sqlite3 pavyzdys.db .dump > export.sql` sukuria SQL failą. Tačiau prieš importuojant į MySQL, būtina redaguoti sintaksę – pvz. pakeisti `AUTOINCREMENT`, kabutes ir pan., kad ji atitiktų MySQL reikalavimus.

**3. Naudoti Python ir perkelti su `pandas`:**  
Galima nuskaityti SQLite duomenis į `DataFrame`, o tada įrašyti į MySQL naudojant `to_sql()` su `sqlalchemy` ir `pymysql` jungtimi.

SQLite puikiai tinka lokaliam darbui, bet jei reikia dirbti toliau MySQL Workbench'e – duomenis reikia eksportuoti ir perkelti naudojant vieną iš aukščiau pateiktų metodų.
